# Load Depenency

In [15]:
#Cleaned up Crypto Example
from os import system
import pandas as pd
import requests
import datetime as dt
# from pytrials.client import ClinicalTrials
import json
import ipywidgets as widgets
import numpy as np
np.random.seed(10031975)
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [16]:
#does some error handling


import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [17]:
from pytrials.utils import json_handler, csv_handler


class ClinicalTrials:
    """ClinicalTrials API client

    Provides functions to easily access the ClinicalTrials.gov API
    (https://clinicaltrials.gov/api/)
    in Python.

    Attributes:
        study_fields: List of all study fields you can use in your query.
        api_info: Tuple containing the API version number and the last
        time the database was updated.
    """

    _BASE_URL = "https://clinicaltrials.gov/api/"
    _INFO = "info/"
    _QUERY = "query/"
    _JSON = "fmt=json"
    _CSV = "fmt=csv"

    def __init__(self):
        self.api_info = self.__api_info()

    @property
    def study_fields(self):
        fields_list = json_handler(
            f"{self._BASE_URL}{self._INFO}study_fields_list?{self._JSON}"
        )
        return fields_list["StudyFields"]["Fields"]

    def __api_info(self):
        """Returns information about the API"""
        last_updated = json_handler(
            f"{self._BASE_URL}{self._INFO}data_vrs?{self._JSON}"
        )["DataVrs"]
        api_version = json_handler(f"{self._BASE_URL}{self._INFO}api_vrs?{self._JSON}")[
            "APIVrs"
        ]

        return api_version, last_updated

    def get_full_studies(self, search_expr, min_rnk=1, max_studies=100):
        """Returns all content for a maximum of 100 study records.

        Retrieves information from the full studies endpoint, which gets all study fields.
        This endpoint can only output JSON (Or not-supported XML) format and does not allow
        requests for more than 100 studies at once.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.

        Returns:
            dict: Object containing the information queried with the search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 100
        """
        if max_studies > 100 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 100")

        req = f"full_studies?expr={search_expr}&min_rnk={min_rnk}&max_rnk={max_studies+min_rnk-1}&{self._JSON}"

        full_studies = json_handler(f"{self._BASE_URL}{self._QUERY}{req}")

        return full_studies

    def get_study_fields(self, search_expr, fields, max_studies=50, min_rnk=1,fmt="csv"):
        """Returns study content for specified fields

        Retrieves information from the study fields endpoint, which acquires specified information
        from a large (max 1000) studies. To see a list of all possible fields, check the class'
        study_fields attribute.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            fields (list(str)): A list containing the desired information fields.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.
            fmt (str): A string indicating the output format, csv or json. Defaults to csv.

        Returns:
            Either a dict, if fmt='json', or a list of records (e.g. a list of lists), if fmt='csv.
            Both containing the maximum number of study fields queried using the specified search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 1000
            ValueError: One of the fields is not valid! Check the study_fields attribute
                for a list of valid ones.
            ValueError: Format argument has to be either 'csv' or 'json'
        """
        if max_studies > 1000 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 1000")
        elif not set(fields).issubset(self.study_fields):
            raise ValueError(
                "One of the fields is not valid! Check the study_fields attribute for a list of valid ones."
            )
        else:
            concat_fields = ",".join(fields)
            # req = f"study_fields?expr={search_expr}&max_rnk={max_studies}&fields={concat_fields}"
            req = f"study_fields?expr={search_expr}&min_rnk={min_rnk}&max_rnk={max_studies+min_rnk-1}&fields={concat_fields}"
            if fmt == "csv":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._CSV}"
                return csv_handler(url)

            elif fmt == "json":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
                return json_handler(url)

            else:
                raise ValueError("Format argument has to be either 'csv' or 'json'")

    def get_study_count(self, search_expr):
        """Returns study count for specified search expression

        Retrieves the count of studies matching the text entered in search_expr.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.

        Returns:
            An integer

        Raises:
            ValueError: The search expression cannot be blank.
        """
        if not set(search_expr):
            raise ValueError("The search expression cannot be blank.")
        else:
            req = f"study_fields?expr={search_expr}&max_rnk=1&fields=NCTId"
            url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
            returned_data = json_handler(url)
            study_count = returned_data["StudyFieldsResponse"]["NStudiesFound"]
            return study_count

    def __repr__(self):
        return f"ClinicalTrials.gov client v{self.api_info[0]}, database last updated {self.api_info[1]}"

# Load Data

https://clinicaltrials.gov/api/gui/ref/crosswalks

Disease 2 search for

This is for dymanic input of diease searching

In [18]:
# disease=input()
# print(disease)

In [19]:
disease = 'TMS'

#disease = 'depression'

# disease = 'diabetes'

# disease = 'tacs'



hit mesh some day

Pull ONE

#Explore Data

In [20]:
#change this out to NAME

ct = ClinicalTrials()
infodf= (ct.get_full_studies(search_expr=disease, max_studies=5 ))


In [21]:
type(infodf)

dict

In [22]:
number = print(infodf['FullStudiesResponse']['NStudiesFound'])

2339


In [23]:
count = ct.get_study_count(search_expr=disease)


In [24]:
# count = 3000
start = 0
end = count
iter = 99 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df = pd.DataFrame()

array = {}
array ['key'] = 'value'
json_data = json.dumps(array)

for i in range(1, count, iter):
    # print(temp)
    temp =ct.get_full_studies(
        search_expr=disease,
        max_studies=iter,
        min_rnk=i,
        
        # fmt="csv",
        )
    # print(count),
    print(str(i) + "-" + str(iter) + " of " + str(count))
    # print(temp2[0])
    temp2 =  pd.json_normalize(temp)
    # json_object = temp['FullStudiesResponse']
    # array = temp['FullStudiesResponse']
    # temp2 = pd.json_normalize(temp())
    new_df = pd.concat([new_df, temp2])
 
# https://clinicaltrials.gov/api/gui/demo/simple_full_study

1-99 of 2339
100-99 of 2339
199-99 of 2339
298-99 of 2339
397-99 of 2339
496-99 of 2339
595-99 of 2339
694-99 of 2339
793-99 of 2339
892-99 of 2339
991-99 of 2339
1090-99 of 2339
1189-99 of 2339
1288-99 of 2339
1387-99 of 2339
1486-99 of 2339
1585-99 of 2339
1684-99 of 2339
1783-99 of 2339
1882-99 of 2339
1981-99 of 2339
2080-99 of 2339
2179-99 of 2339
2278-99 of 2339


In [25]:
print(type(new_df))

<class 'pandas.core.frame.DataFrame'>


In [26]:
new_df.to_excel(disease+'_data_raw.xlsx', index = False)

In [27]:
# with open(disease+'data2.json', 'w', encoding='utf-8') as f:
#     json.dump(new_df, f, ensure_ascii=False, indent=4)

In [28]:
# count = 3000
start = 0
end = count
iter = 99 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df = pd.DataFrame()

temp = ct.get_full_studies(
        search_expr='tacs',
    #     fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    # "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BaselineMeasureTitle","BaselineMeasureDescription","BaselineMeasurementValue","BriefSummary"],
        max_studies=iter,
        min_rnk=5,
        # fmt="csv",
        )
# print(full_studies)
# print(temp)

    # temp2 =  pd.DataFrame.from_records(temp[1:])
    # new_df = pd.concat([new_df, temp2])
# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(temp, f, ensure_ascii=False, indent=4)

# number = print(temp['FullStudiesResponse'])

In [29]:
new_df.columns = ["Index","NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BaselineMeasureTitle","BaselineMeasureDescription","BaselineMeasurementValue","BriefSummary"]

ValueError: Length mismatch: Expected axis has 0 elements, new values have 20 elements

'NStudiesFound': 3656,

In [ ]:
# new_df
new_df.shape

In [ ]:
new_df.head

In [ ]:
# temp = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])

In [ ]:
# print(temp.shape)

In [ ]:
# count = 3000
start = 0
end = count
iter = 999 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df2 = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df2 = pd.concat([new_df2, temp2])
 


In [ ]:
new_df2.columns = ["index", "NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"]

In [ ]:
new_df2.shape

In [ ]:
new_df2

In [ ]:




# api_pull_2 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId"],
#     max_studies=10,
#     min_rnk=5,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records



# # Read the csv data in Pandas
# import pandas as pd

# temp = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])
# temp

In [ ]:




# # Get the NCTId, Condition and Brief title fields from 500 studies related to Coronavirus and Covid, in csv format.
# api_pull_1 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
#     "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" 
#     ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","IsFDARegulatedDrug","IsFDARegulatedDevice","BriefSummary"],
#     max_studies=999,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records



# # Read the csv data in Pandas
# import pandas as pd

# df1=pd.DataFrame.from_records(api_pull_1[1:], columns=api_pull_1[0])

# # 
# # df1 = pd.DataFrame.to_frame().reset_index()



In [ ]:
# api_pull_2 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
# ,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"

# ],
#     max_studies=999,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records

# ct.get_study_count(search_expr="Parkinson")

# # Read the csv data in Pandas
# import pandas as pd

# df2 = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])

##cleaning

In [ ]:
# print(type(df1))

In [ ]:
result = pd.merge(new_df, new_df2,  on='NCTId', how='outer')

In [ ]:
result.shape

In [ ]:
# result = result.loc[:,~result.columns.duplicated()]

In [ ]:
# result.shape

In [ ]:
# result.columns.tolist()

IMPORTANT - this just drops the second index column - monitor or watch for mismatch or if you change the number of elements that are pulled from the first api

In [ ]:
result = result.drop(result.columns[[20]], axis=1)

In [ ]:
result.shape

In [ ]:
# result.columns.tolist()

Save file

In [ ]:
result.to_csv(disease+'-json_data_raw.csv', index = False)
result.to_excel(disease+'-json_data_raw.xlsx', index = False)

In [ ]:
new_df.to_hdf('data_raw.h5', key='df', mode='w')  

In [ ]:
# labels, levels = pd.factorize(result["Phase"])  

In [ ]:
# labels

In [ ]:
# fun_barplot()